## Intro

Object-oriented programming is a way of modeling programs that involving objects which pass messages to each other.

In this chaper, we will learn to implement an object-oriented design pattern in Rust and discuss the trade-offs of doing so versus implementing a solution using some of Rust’s strengths instead.

## Characteristics of Object-Oriented Languages

### Objects Contain Data and Behavior

In the Gang of Four book, OOP is defined as:

> <span style="color:lightgreen">***Object-oriented programs are made up of objects. An object packages both data and the procedures that operate on that data. The procedures are typically called methods or operations.***</span>

Using this definition, <span style="color:lightgreen">*Rust is object-oriented: structs and enums have data, and `impl` blocks provide methods on structs and enums.*</span>

### Encapsulation that Hides Implementation Details

Another aspect commonly associated with OOP is the idea of encapsulation, meaning:

> <span style="color:lightgreen">***Encapsulation: the implementation details of an object aren’t accessible to code using that object. Therefore, the only way to interact with an object is through its public API; code using the object shouldn’t be able to reach into the object’s internals and change data or behavior directly.***</span>

**Example**: Below, we have an `AveragedCollection` struct is marked `pub` so that other code can use it, but the fields within the struct remain private. We have public methods which are `add`, `remove`, and `average` which are the only ways to access or modify data in an instance of `AveragedCollection`. We leave the `list` and `average` fields private so there is no way for external code to add or remove items to or from the `list` field directly; otherwise, the `average` field might become out of sync when the list changes.

```rust
pub struct AveragedCollection {
    list: Vec<i32>,
    average: f64,
}


impl AveragedCollection {
    pub fn add(&mut self, value: i32) {
        self.list.push(value);
        self.update_average();
    }

    pub fn remove(&mut self) -> Option<i32> {
        let result = self.list.pop();
        match result {
            Some(value) => {
                self.update_average();
                Some(value)
            }
            None => None,
        }
    }

    pub fn average(&self) -> f64 {
        self.average
    }

    fn update_average(&mut self) {
        let total: i32 = self.list.iter().sum();
        self.average = total as f64 / self.list.len() as f64;
    }
}impl AveragedCollection {
    pub fn add(&mut self, value: i32) {
        self.list.push(value);
        self.update_average();
    }

    pub fn remove(&mut self) -> Option<i32> {
        let result = self.list.pop();
        match result {
            Some(value) => {
                self.update_average();
                Some(value)
            }
            None => None,
        }
    }

    pub fn average(&self) -> f64 {
        self.average
    }

    fn update_average(&mut self) {
        let total: i32 = self.list.iter().sum();
        self.average = total as f64 / self.list.len() as f64;
    }
}
```

### Inheritance as a Type System and as Code Sharing

> <span style="color:lightgreen">***Inheritance is a mechanism whereby an object can inherit elements from another object’s definition, thus gaining the parent object’s data and behavior without you having to define them again.***</span>

Why using inheritance?
- <span style="color:lightgreen">*Reuse of code*</span>: you can implement particular behavior for one type, and inheritance enables you to reuse that implementation for a different type. You can do this in a limited way in Rust code using default `trait` method implementations
- <span style="color:lightgreen">*Use inheritance relates to the type system*</span>: to enable a child type to be used in the same places as the parent type.  This is also called <span style="color:lightgreen">***polymorphism***</span>, which means that you can substitute multiple objects for each other at runtime if they share certain characteristics.

#### Polymorphism

> To many people, polymorphism is synonymous with inheritance. But it’s actually a more general concept that refers to <span style="color:lightgreen">*code that can work with data of multiple types. For inheritance, those types are generally subclasses.*</span>

> <span style="color:skyblue">*Rust instead uses generics to abstract over different possible types and trait bounds to impose constraints on what those types must provide. This is sometimes called bounded parametric polymorphism.*</span>


### Inheritance Cons

<span style="color:orange">*Inheritance has recently fallen out of favor as a programming design solution in many programming languages because it’s often at risk of sharing more code than necessary. Subclasses shouldn’t always share all characteristics of their parent class but will do so with inheritance. This can make a program’s design less flexible. It also introduces the possibility of calling methods on subclasses that don’t make sense or that cause errors because the methods don’t apply to the subclass. In addition, some languages will only allow single inheritance (meaning a subclass can only inherit from one class), further restricting the flexibility of a program’s design.*</span>

<span style="color:skyblue">*For these reasons, Rust takes the different approach of **using trait objects instead of inheritance**. Let’s look at how trait objects enable polymorphism in Rust.*</span>

## Using Trait Objects That Allow for Values of Different Types

Example: Building a GUI library  
In a language with inheritance, we might define a class named `Component` that has a method named `draw` on it. The other classes, such as `Button`, `Image`, and `SelectBox`, would inherit from `Component` and thus inherit the `draw` method. They could each override the dra`w` method to define their custom behavior, but the framework could treat all of the types as if they were `Component` instances and call `draw` on them. But because Rust doesn’t have inheritance, we need another way to structure the gui library to allow users to extend it with new types.

To implement the behavior we want `gui` to have, we’ll define a trait named `Draw` that will have one method named `draw`. Then we can define a vector that takes a trait object. <span style="color:lightgreen">*A trait object points to both an instance of a type implementing our specified trait and a table used to look up trait methods on that type at runtime. We create a trait object by specifying some sort of pointer, such as a `&` reference or a `Box<T>` smart pointer, then the `dyn` keyword, and then specifying the relevant trait.*</span>

In [2]:
// definition of the Draw trait
pub trait Draw {
    fn draw(&self);
}

// Definition of the `Screen` struct with a `components` field holding a 
// vector of trait objects that implement the `Draw` trait
pub struct Screen {
    pub components: Vec<Box<dyn Draw>>,
    // By specifying `Box<dyn Draw>` as the type of the values in 
    // the components vector, we’ve defined `Screen` to need values 
    // that we can call the `draw` method on.
}

// On the Screen struct, we’ll define a method named `run` that will 
// call the dr`aw method on each of its components
impl Screen {
    pub fn run(&self) {
        for component in self.components.iter() {
            component.draw();
        }
    }
}

// Definition of the `Button` struct with `width`, `height`, and `label` fields
// A Button struct that implements the `Draw` trait
pub struct Button {
    pub width: u32,
    pub height: u32,
    pub label: String,
}

impl Draw for Button {
    fn draw(&self) {
        // code to actually draw a button
        println!("drawing Button");
    }
}

// Another crate using `gui` and implementing the `Draw` trait on a `SelectBox` struct
struct SelectBox {
    width: u32,
    height: u32,
    options: Vec<String>,
}

impl Draw for SelectBox {
    fn draw(&self) {
        // code to actually draw a select box
        println!("drawing SelectBox");
    }
}

fn main() {
    let screen = Screen {
        components: vec![
            Box::new(SelectBox {
                width: 75,
                height: 10,
                options: vec![
                    String::from("Yes"),
                    String::from("Maybe"),
                    String::from("No"),
                ],
            }),
            Box::new(Button {
                width: 50,
                height: 10,
                label: String::from("OK"),
            }),
        ],
    };

    screen.run();
}

main()

drawing SelectBox
drawing Button


()

> <span style="color:skyblue">*`Screen.run` doesn’t need to know what the concrete type of each component is. It doesn’t check whether a component is an instance of a `Button` or a `SelectBox`, it just calls the `draw` method on the component. This concept of being concerned only with the messages a value responds to rather than the value’s concrete type—is similar to the concept of **duck typing** in dynamically typed languages: if it walks like a duck and quacks like a duck, then it must be a duck*</span>. <span style="color:lightgreen">*The advantage of using trait objects and Rust’s type system to write code similar to code using duck typing is that we never have to check whether a value implements a particular method at runtime or worry about getting errors if a value doesn’t implement a method but we call it anyway. Rust won’t compile our code if the values don’t implement the traits that the trait objects need.*</span>

In other languages, the data and behavior combined into one concept is often labeled an object. However, <span style="color:lightgreen">*trait objects are more like objects in other languages in the sense that they combine data and behavior. But trait objects differ from traditional objects in that we can’t add data to a trait object.*</span>

This works differently from defining a struct that uses a generic type parameter with trait bounds. <span style="color:orange">*A generic type parameter can only be substituted with one concrete type at a time, whereas trait objects allow for multiple concrete types to fill in for the trait object at runtime.*</span>:

In [ ]:
// An alternate implementation of the `Screen` struct and 
// its `run` method using generics and trait bounds

pub struct Screen<T: Draw> {
    pub components: Vec<T>,
}

impl<T> Screen<T>
where
    T: Draw,
{
    pub fn run(&self) {
        for component in self.components.iter() {
            component.draw();
        }
    }
}

## Implementing an Object-Oriented Design Pattern

<span style="color:lightgreen">*The **state pattern** is an object-oriented design pattern. The crux of the pattern is that we **define a set of states a value can have internally**. The states are represented by a set of state objects, and the value’s behavior changes based on its state.*</span> The advantage of using the state pattern is that, when the business requirements of the program change, we won’t need to change the code of the value holding the state or the code that uses the value. We’ll only need to update the code inside one of the state objects to change its rules or perhaps add more state objects.

- **Example**: We’re going to work through an example of *a blog post struct that has a field to hold its state, which will be a state object from the set "draft", "review", or "published".* The final functionality will look like this:
    - A blog post starts as an empty draft.
    - When the draft is done, a review of the post is requested.
    - When the post is approved, it gets published.
    - Only published blog posts return content to print, so unapproved posts can’t accidentally be published.

In [ ]:
// ------------ main.rs ---------------

fn main() {
    let mut post = Post::new();  // create a new draft blog post

    post.add_text("I ate a salad for lunch today"); // add text to the blog popt
    assert_eq!("", post.content()); // check that the content of the blog post is empty

    post.request_review();  // enable a request for a review of the post,
    assert_eq!("", post.content()); // check that the content of the blog post is still empty

    post.approve();  // approve the post
    assert_eq!("I ate a salad for lunch today", post.content()); // check that the content of the blog post 
                                                    // is now the text we added
}

// ------------ lib.rs ---------------
// Defining Post and Creating a New Instance in the Draft State
pub struct Post {
    state: Option<Box<dyn State>>,
    content: String,
}

impl Post {
    pub fn new() -> Post {
        Post {
            state: Some(Box::new(Draft {})),
            content: String::new(),
        }
    }

    pub fn add_text(&mut self, text: &str) {
        // The add_text method takes a mutable reference to self, 
        // because we’re changing the Post instance that we’re calling add_text on
        self.content.push_str(text)
    }

    pub fn content(&self) -> &str {
        ""
    }
}

trait State{} // The State trait defines the behavior shared by different post states.


// The state objects are Draft, PendingReview, and Published, and they will all implement the State trait
struct Draft {}

impl State for Draft

Notice that the only type we’re interacting with from the crate is the `Post`. `Post` uses the state pattern and will hold a value that will be one of three state objects representing the various states a post can be in—draft, waiting for review, or published.